# Chapter 12: File I/O Operations for VLSI Automation 📁

## 🎯 **Learning Objectives**
Master Python file operations for professional VLSI automation:

### **Core File Operations**
- Reading and writing text files (Verilog, SDC, reports)
- Binary file handling and data processing
- Path manipulation and directory operations
- File system navigation and management

### **VLSI File Processing**
- **Design Files**: Verilog parsing and generation
- **Constraint Files**: SDC reading and writing
- **Report Processing**: Timing, power, area analysis
- **Log Analysis**: Tool output parsing and metrics extraction

### **Advanced Techniques**
- Error handling and file validation
- Large file processing strategies
- Data transformation and format conversion
- Backup and versioning workflows

---

## 🔧 **Why File I/O Matters in VLSI**
File operations are fundamental to VLSI automation:
- **Tool Integration**: Processing input/output files from EDA tools
- **Data Exchange**: Converting between different file formats
- **Report Analysis**: Extracting metrics from tool reports
- **Script Automation**: Reading configurations and generating outputs
- **Quality Control**: Validating design files and catching errors

In [ ]:
# BASIC FILE OPERATIONS FOR VLSI DESIGN
# =====================================
# Essential file reading and writing techniques

print("📁 BASIC FILE OPERATIONS FOR VLSI DESIGN")
print("=" * 45)

import os
import tempfile
from pathlib import Path

# =============================================================================
# TEXT FILE READING AND WRITING
# =============================================================================

print("\n📝 TEXT FILE READING AND WRITING:")

# Sample VLSI file contents
verilog_content = """module cpu_core (
    input wire clk,
    input wire reset_n,
    input wire [31:0] instruction,
    output reg [31:0] result
);

reg [31:0] register_file [0:31];
reg [31:0] pc;

always @(posedge clk or negedge reset_n) begin
    if (!reset_n) begin
        pc <= 32'h0;
        result <= 32'h0;
    end else begin
        case (instruction[31:26])
            6'b000001: result <= register_file[instruction[25:21]] + register_file[instruction[20:16]];
            6'b000010: result <= register_file[instruction[25:21]] - register_file[instruction[20:16]];
            default: result <= 32'h0;
        endcase
        pc <= pc + 4;
    end
end

endmodule"""

sdc_content = """# Timing constraints for cpu_core
create_clock -name clk -period 10.0 [get_ports clk]
set_input_delay -clock clk -max 2.0 [get_ports instruction]
set_input_delay -clock clk -min 0.5 [get_ports instruction]
set_output_delay -clock clk -max 3.0 [get_ports result]
set_output_delay -clock clk -min 0.5 [get_ports result]
set_max_area 1500.0
set_max_dynamic_power 0.8"""

# Create temporary directory for examples
with tempfile.TemporaryDirectory() as temp_dir:
    temp_path = Path(temp_dir)

    # =============================================================================
    # WRITING FILES - DIFFERENT METHODS
    # =============================================================================

    print("   Writing VLSI files:")

    # Method 1: Basic write with open()
    verilog_file = temp_path / "cpu_core.v"
    with open(verilog_file, 'w') as f:
        f.write(verilog_content)
    print(f"     ✅ Verilog file: {verilog_file.name} ({len(verilog_content)} chars)")

    # Method 2: Using Path.write_text()
    sdc_file = temp_path / "constraints.sdc"
    sdc_file.write_text(sdc_content)
    print(f"     ✅ SDC file: {sdc_file.name} ({len(sdc_content.splitlines())} lines)")

    # Method 3: Writing with encoding specification
    report_file = temp_path / "synthesis_report.txt"
    report_content = """
Synthesis Report for cpu_core
============================
Area: 1250.5 μm²
Power: 0.825 W
Frequency: 1000 MHz
Timing: MET (slack: +0.123ns)
    """
    with open(report_file, 'w', encoding='utf-8') as f:
        f.write(report_content.strip())
    print(f"     ✅ Report file: {report_file.name} ({report_file.stat().st_size} bytes)")

    # =============================================================================
    # READING FILES - DIFFERENT METHODS
    # =============================================================================

    print(f"\n📖 READING FILES - DIFFERENT METHODS:")

    # Method 1: Read entire file
    print("   Method 1 - Read entire file:")
    with open(verilog_file, 'r') as f:
        content = f.read()
        module_count = content.count('module')
        always_count = content.count('always')
        print(f"     Verilog analysis: {module_count} modules, {always_count} always blocks")

    # Method 2: Read line by line
    print("\n   Method 2 - Line by line processing:")
    constraint_count = 0
    comment_count = 0
    with open(sdc_file, 'r') as f:
        for line_num, line in enumerate(f, 1):
            line = line.strip()
            if line.startswith('#'):
                comment_count += 1
            elif line and not line.startswith('#'):
                constraint_count += 1
        print(f"     SDC analysis: {constraint_count} constraints, {comment_count} comments in {line_num} lines")

    # Method 3: Read all lines into list
    print("\n   Method 3 - Read all lines:")
    with open(report_file, 'r') as f:
        lines = f.readlines()

    # Process report lines
    metrics = {}
    for line in lines:
        line = line.strip()
        if ':' in line and any(keyword in line for keyword in ['Area', 'Power', 'Frequency', 'Timing']):
            parts = line.split(':')
            if len(parts) == 2:
                key = parts[0].strip()
                value = parts[1].strip()
                metrics[key] = value

    print(f"     Report metrics extracted:")
    for key, value in metrics.items():
        print(f"       {key}: {value}")

    # =============================================================================
    # FILE INFORMATION AND OPERATIONS
    # =============================================================================

    print(f"\n📊 FILE INFORMATION AND OPERATIONS:")

    for file_path in [verilog_file, sdc_file, report_file]:
        stat = file_path.stat()
        print(f"   {file_path.name}:")
        print(f"     Size: {stat.st_size} bytes")
        print(f"     Exists: {file_path.exists()}")
        print(f"     Is file: {file_path.is_file()}")
        print(f"     Readable: {os.access(file_path, os.R_OK)}")
        print(f"     Extension: {file_path.suffix}")

print("\n🏆 BASIC FILE I/O BENEFITS:")
print("✅ **Design File Processing**: Read/write Verilog, SDC, reports")
print("✅ **Automated Analysis**: Extract metrics from tool outputs")
print("✅ **Cross-Platform**: Path operations work on all systems")
print("✅ **Flexible Methods**: Multiple approaches for different needs")

In [ ]:
# ADVANCED FILE PROCESSING TECHNIQUES
# ===================================
# CSV, JSON, and structured data handling

print("🔧 ADVANCED FILE PROCESSING TECHNIQUES")
print("=" * 40)

import csv
import json
import re
from datetime import datetime

# =============================================================================
# CSV PROCESSING FOR DESIGN DATA
# =============================================================================

print("\n📈 CSV PROCESSING FOR DESIGN DATA:")

# Function to write design metrics to CSV
def write_design_metrics_csv(filename, design_data):
    """Write design metrics to CSV file."""
    with open(filename, 'w', newline='') as f:
        writer = csv.writer(f)
        # Write header
        writer.writerow(['Module', 'Area_um2', 'Power_mW', 'Frequency_MHz', 'Utilization', 'Timing_Slack'])
        # Write data
        for row in design_data:
            writer.writerow(row)
    return len(design_data)

# Function to read and analyze CSV data
def read_design_metrics_csv(filename):
    """Read and analyze design metrics from CSV file."""
    designs = []
    with open(filename, 'r') as f:
        reader = csv.DictReader(f)
        for row in reader:
            design = {
                'module': row['Module'],
                'area': float(row['Area_um2']),
                'power': float(row['Power_mW']),
                'frequency': float(row['Frequency_MHz']),
                'utilization': float(row['Utilization']),
                'slack': float(row['Timing_Slack'])
            }
            designs.append(design)
    return designs

# Function to analyze design data
def analyze_design_data(designs):
    """Analyze design metrics and return summary."""
    total_area = sum(d['area'] for d in designs)
    total_power = sum(d['power'] for d in designs)
    failed_timing = [d for d in designs if d['slack'] < 0]
    avg_utilization = sum(d['utilization'] for d in designs) / len(designs)

    return {
        'total_modules': len(designs),
        'total_area': total_area,
        'total_power': total_power,
        'failed_timing': len(failed_timing),
        'failed_modules': [d['module'] for d in failed_timing],
        'avg_utilization': avg_utilization
    }

# Sample design data
design_data = [
    ['cpu_core', 1250.5, 825.0, 1000, 0.75, 0.123],
    ['memory_ctrl', 890.2, 456.0, 800, 0.68, 0.089],
    ['io_interface', 234.8, 125.0, 100, 0.45, 0.567],
    ['crypto_unit', 678.9, 234.0, 500, 0.82, -0.045],
    ['dsp_block', 445.6, 178.0, 200, 0.59, 0.234]
]

with tempfile.TemporaryDirectory() as temp_dir:
    temp_path = Path(temp_dir)
    csv_file = temp_path / "design_metrics.csv"

    # Write and read CSV
    print("   CSV processing workflow:")
    rows_written = write_design_metrics_csv(csv_file, design_data)
    print(f"     ✅ Written {rows_written} design records to CSV")

    designs = read_design_metrics_csv(csv_file)
    print(f"     ✅ Read {len(designs)} design records from CSV")

    # Analyze data
    analysis = analyze_design_data(designs)
    print(f"\n   Design analysis results:")
    print(f"     Total modules: {analysis['total_modules']}")
    print(f"     Total area: {analysis['total_area']:.1f} μm²")
    print(f"     Total power: {analysis['total_power']:.1f} mW")
    print(f"     Average utilization: {analysis['avg_utilization']:.1%}")
    print(f"     Timing violations: {analysis['failed_timing']} modules")
    if analysis['failed_modules']:
        print(f"     Failed modules: {analysis['failed_modules']}")

    # =============================================================================
    # JSON PROCESSING FOR CONFIGURATION DATA
    # =============================================================================

    print(f"\n🔧 JSON PROCESSING FOR CONFIGURATION DATA:")

    # Function to create tool configuration
    def create_tool_config():
        """Create comprehensive tool configuration."""
        return {
            "project": {
                "name": "cpu_design",
                "version": "1.0",
                "technology": "28nm",
                "created": datetime.now().isoformat()
            },
            "synthesis": {
                "tool": "Design Compiler",
                "version": "2023.03",
                "settings": {
                    "optimization_level": "high",
                    "area_effort": "medium",
                    "timing_effort": "high",
                    "power_effort": "low"
                },
                "libraries": [
                    "/libs/tsmc28/timing.lib",
                    "/libs/tsmc28/power.lib"
                ],
                "constraints": {
                    "max_area": 1500.0,
                    "target_frequency": 1000,
                    "max_power": 1.0
                }
            },
            "place_route": {
                "tool": "Innovus",
                "version": "21.1",
                "settings": {
                    "placement_effort": "standard",
                    "routing_effort": "high",
                    "optimization": "timing"
                },
                "technology": {
                    "process": "28nm",
                    "metal_layers": 9,
                    "min_width": 0.09
                }
            }
        }

    # Function to validate configuration
    def validate_tool_config(config):
        """Validate tool configuration."""
        errors = []

        # Check required fields
        if 'project' not in config:
            errors.append("Missing project information")

        if 'synthesis' in config:
            syn_config = config['synthesis']
            if 'constraints' in syn_config:
                constraints = syn_config['constraints']
                if constraints.get('target_frequency', 0) <= 0:
                    errors.append("Invalid target frequency")
                if constraints.get('max_area', 0) <= 0:
                    errors.append("Invalid max area constraint")

        return errors

    # Function to extract tool summary
    def extract_tool_summary(config):
        """Extract summary information from configuration."""
        summary = {}

        if 'project' in config:
            summary['project'] = config['project']['name']
            summary['technology'] = config['project']['technology']

        for tool_name in ['synthesis', 'place_route']:
            if tool_name in config:
                tool_config = config[tool_name]
                summary[tool_name] = {
                    'tool': tool_config.get('tool', 'Unknown'),
                    'version': tool_config.get('version', 'Unknown')
                }

        return summary

    json_file = temp_path / "tool_config.json"

    # Create and save configuration
    config = create_tool_config()
    with open(json_file, 'w') as f:
        json.dump(config, f, indent=2)
    print(f"     ✅ Configuration saved to {json_file.name}")

    # Read and validate configuration
    with open(json_file, 'r') as f:
        loaded_config = json.load(f)

    errors = validate_tool_config(loaded_config)
    if errors:
        print(f"     ❌ Configuration errors: {errors}")
    else:
        print(f"     ✅ Configuration validation passed")

    # Extract and display summary
    summary = extract_tool_summary(loaded_config)
    print(f"\n   Configuration summary:")
    print(f"     Project: {summary.get('project', 'N/A')} ({summary.get('technology', 'N/A')})")
    for tool, info in summary.items():
        if tool not in ['project', 'technology']:
            print(f"     {tool.title()}: {info['tool']} v{info['version']}")

print("\n🏆 ADVANCED FILE PROCESSING BENEFITS:")
print("✅ **Structured Data**: CSV/JSON for design metrics and configurations")
print("✅ **Data Validation**: Configuration and format checking")
print("✅ **Tool Integration**: Standard formats for data exchange")
print("✅ **Automation Ready**: Programmatic data processing")

In [ ]:
# VLSI REPORT PARSING AND ANALYSIS
# ================================
# Extracting metrics from EDA tool reports

print("📊 VLSI REPORT PARSING AND ANALYSIS")
print("=" * 40)

import re
from collections import defaultdict

# =============================================================================
# TIMING REPORT PARSING
# =============================================================================

print("\n⏱️ TIMING REPORT PARSING:")

# Sample timing report content
timing_report_content = """
****************************************
Report : timing
        -path full
        -delay max
        -max_paths 10
Design : cpu_core
Version: X-2021.09
Date   : Sat Sep  7 10:30:00 2025
****************************************

Startpoint: instruction[0] (input port clocked by clk)
Endpoint: result[31] (output port clocked by clk)
Path Group: clk
Path Type: max

  Delay    Time   Description
-----------------------------------------
   0.00    0.00   clock clk (rise edge)
   2.00    2.00   input external delay
   0.15    2.15 ^ instruction[0] (in)
   1.25    3.40 ^ U15/Y (INVX1)
   0.85    4.25 v U23/Y (NAND2X1)
   1.35    5.60 ^ U45/Y (NOR2X1)
   0.95    6.55 v result[31] (out)
   3.00    9.55   output external delay
  10.00   10.00   clock clk (rise edge)
           0.45   clock uncertainty
  10.45   10.45   clock reconvergence pessimism
-----------------------------------------
  10.45          required time
   9.55          arrival time
-----------------------------------------
   0.90          slack (MET)

Startpoint: instruction[8] (input port clocked by clk)
Endpoint: result[15] (output port clocked by clk)
Path Group: clk
Path Type: max

  Delay    Time   Description
-----------------------------------------
   0.00    0.00   clock clk (rise edge)
   2.00    2.00   input external delay
   0.15    2.15 ^ instruction[8] (in)
   1.45    3.60 ^ U25/Y (INVX2)
   0.95    4.55 v U33/Y (NAND2X1)
   1.15    5.70 ^ U55/Y (NOR2X1)
   0.85    6.55 v result[15] (out)
   3.00    9.55   output external delay
  10.00   10.00   clock clk (rise edge)
           0.45   clock uncertainty
  10.45   10.45   clock reconvergence pessimism
-----------------------------------------
  10.45          required time
   9.55          arrival time
-----------------------------------------
   0.90          slack (MET)
"""

def parse_timing_report(report_content):
    """Parse timing report and extract key metrics."""
    lines = report_content.strip().split('\n')

    # Extract header information
    header_info = {}
    design_match = re.search(r'Design\s*:\s*(\w+)', report_content)
    if design_match:
        header_info['design'] = design_match.group(1)

    # Extract timing paths
    paths = []
    current_path = None
    in_path_section = False

    for line in lines:
        line = line.strip()

        # Detect start of new path
        if line.startswith('Startpoint:'):
            if current_path:
                paths.append(current_path)
            current_path = {
                'startpoint': re.search(r'Startpoint:\s*([^\s]+)', line).group(1),
                'endpoint': '',
                'path_group': '',
                'path_type': '',
                'arrival_time': 0.0,
                'required_time': 0.0,
                'slack': 0.0,
                'status': 'UNKNOWN'
            }

        elif line.startswith('Endpoint:') and current_path:
            current_path['endpoint'] = re.search(r'Endpoint:\s*([^\s]+)', line).group(1)

        elif line.startswith('Path Group:') and current_path:
            current_path['path_group'] = re.search(r'Path Group:\s*(\w+)', line).group(1)

        elif line.startswith('Path Type:') and current_path:
            current_path['path_type'] = re.search(r'Path Type:\s*(\w+)', line).group(1)

        elif 'arrival time' in line and current_path:
            time_match = re.search(r'([\d.-]+)\s+arrival time', line)
            if time_match:
                current_path['arrival_time'] = float(time_match.group(1))

        elif 'required time' in line and current_path:
            time_match = re.search(r'([\d.-]+)\s+required time', line)
            if time_match:
                current_path['required_time'] = float(time_match.group(1))

        elif 'slack' in line and current_path:
            slack_match = re.search(r'([\d.-]+)\s+slack\s*\((\w+)\)', line)
            if slack_match:
                current_path['slack'] = float(slack_match.group(1))
                current_path['status'] = slack_match.group(2)

    # Add last path
    if current_path:
        paths.append(current_path)

    return {
        'header': header_info,
        'paths': paths,
        'total_paths': len(paths)
    }

def analyze_timing_paths(timing_data):
    """Analyze timing paths and generate summary."""
    paths = timing_data['paths']

    if not paths:
        return {'error': 'No timing paths found'}

    # Calculate statistics
    slacks = [path['slack'] for path in paths]
    violations = [path for path in paths if path['slack'] < 0]

    analysis = {
        'total_paths': len(paths),
        'violations': len(violations),
        'worst_slack': min(slacks) if slacks else 0,
        'best_slack': max(slacks) if slacks else 0,
        'average_slack': sum(slacks) / len(slacks) if slacks else 0,
        'timing_status': 'PASS' if len(violations) == 0 else 'FAIL'
    }

    # Find worst path
    if slacks:
        worst_idx = slacks.index(min(slacks))
        analysis['worst_path'] = {
            'startpoint': paths[worst_idx]['startpoint'],
            'endpoint': paths[worst_idx]['endpoint'],
            'slack': paths[worst_idx]['slack']
        }

    return analysis

# Test timing report parsing
print("   Parsing timing report:")
timing_data = parse_timing_report(timing_report_content)
print(f"     ✅ Parsed report for design: {timing_data['header'].get('design', 'Unknown')}")
print(f"     ✅ Found {timing_data['total_paths']} timing paths")

analysis = analyze_timing_paths(timing_data)
print(f"\n   Timing analysis results:")
print(f"     Overall status: {analysis['timing_status']}")
print(f"     Total violations: {analysis['violations']}")
print(f"     Worst slack: {analysis['worst_slack']:.3f} ns")
print(f"     Best slack: {analysis['best_slack']:.3f} ns")
print(f"     Average slack: {analysis['average_slack']:.3f} ns")

if 'worst_path' in analysis:
    worst = analysis['worst_path']
    print(f"     Worst path: {worst['startpoint']} -> {worst['endpoint']} ({worst['slack']:+.3f}ns)")

# =============================================================================
# POWER REPORT PARSING
# =============================================================================

print(f"\n⚡ POWER REPORT PARSING:")

# Sample power report content
power_report_content = """
****************************************
Report : power
        -analysis_effort low
Design : cpu_core
Version: X-2021.09
Date   : Sat Sep  7 10:30:00 2025
****************************************

                Internal         Switching           Leakage            Total
Power Group    Power            Power               Power              Power    (   %  )  Attrs
--------------------------------------------------------------------------------------------------
clock_network  2.341e-02        1.234e-03           1.456e-05          2.465e-02 ( 29.89)
register       1.567e-02        8.912e-04           2.345e-05          1.659e-02 ( 20.11)
combinational  1.234e-02        2.345e-03           3.456e-05          1.472e-02 ( 17.84)
memory         8.901e-03        1.567e-03           1.234e-05          1.048e-02 ( 12.71)
io_pad         6.789e-03        3.456e-04           5.678e-06          7.141e-03 (  8.66)
black_box      4.567e-03        6.789e-04           8.901e-06          5.255e-03 (  6.37)
macro          3.456e-03        5.678e-04           1.234e-05          4.037e-03 (  4.89)
--------------------------------------------------------------------------------------------------
Total          7.456e-02        7.654e-03           1.234e-04          8.245e-02 (100.00)

1
"""

def parse_power_report(report_content):
    """Parse power report and extract power breakdown."""
    lines = report_content.strip().split('\n')

    # Extract header information
    header_info = {}
    design_match = re.search(r'Design\s*:\s*(\w+)', report_content)
    if design_match:
        header_info['design'] = design_match.group(1)

    # Extract power data
    power_groups = []
    total_power = {}

    # Look for data lines with power values
    for line in lines:
        line = line.strip()

        # Skip header and separator lines
        if not line or line.startswith('-') or line.startswith('Power Group') or line.startswith('*'):
            continue

        # Parse data lines with scientific notation
        power_match = re.match(r'(\w+)\s+([\d.e-]+)\s+([\d.e-]+)\s+([\d.e-]+)\s+([\d.e-]+)\s+\(\s*([\d.]+)\)', line)
        if power_match:
            group_name = power_match.group(1)
            internal = float(power_match.group(2))
            switching = float(power_match.group(3))
            leakage = float(power_match.group(4))
            total = float(power_match.group(5))
            percentage = float(power_match.group(6))

            if group_name.lower() == 'total':
                total_power = {
                    'internal': internal,
                    'switching': switching,
                    'leakage': leakage,
                    'total': total
                }
            else:
                power_groups.append({
                    'group': group_name,
                    'internal_power': internal,
                    'switching_power': switching,
                    'leakage_power': leakage,
                    'total_power': total,
                    'percentage': percentage
                })

    return {
        'header': header_info,
        'power_groups': power_groups,
        'total_power': total_power
    }

def analyze_power_breakdown(power_data):
    """Analyze power breakdown and generate insights."""
    groups = power_data['power_groups']
    total = power_data['total_power']

    if not groups or not total:
        return {'error': 'No power data found'}

    # Find top power consumers
    sorted_groups = sorted(groups, key=lambda x: x['total_power'], reverse=True)

    # Calculate power type percentages
    total_power_val = total['total']
    internal_pct = (total['internal'] / total_power_val * 100) if total_power_val > 0 else 0
    switching_pct = (total['switching'] / total_power_val * 100) if total_power_val > 0 else 0
    leakage_pct = (total['leakage'] / total_power_val * 100) if total_power_val > 0 else 0

    analysis = {
        'total_power_mw': total_power_val * 1000,  # Convert to mW
        'power_breakdown': {
            'internal': {'value': total['internal'] * 1000, 'percentage': internal_pct},
            'switching': {'value': total['switching'] * 1000, 'percentage': switching_pct},
            'leakage': {'value': total['leakage'] * 1000, 'percentage': leakage_pct}
        },
        'top_consumers': sorted_groups[:3],
        'group_count': len(groups)
    }

    return analysis

# Test power report parsing
print("   Parsing power report:")
power_data = parse_power_report(power_report_content)
print(f"     ✅ Parsed report for design: {power_data['header'].get('design', 'Unknown')}")
print(f"     ✅ Found {len(power_data['power_groups'])} power groups")

power_analysis = analyze_power_breakdown(power_data)
if 'error' not in power_analysis:
    print(f"\n   Power analysis results:")
    print(f"     Total power: {power_analysis['total_power_mw']:.3f} mW")

    breakdown = power_analysis['power_breakdown']
    print(f"     Power breakdown:")
    print(f"       Internal: {breakdown['internal']['value']:.3f} mW ({breakdown['internal']['percentage']:.1f}%)")
    print(f"       Switching: {breakdown['switching']['value']:.3f} mW ({breakdown['switching']['percentage']:.1f}%)")
    print(f"       Leakage: {breakdown['leakage']['value']:.3f} mW ({breakdown['leakage']['percentage']:.1f}%)")

    print(f"\n     Top power consumers:")
    for i, group in enumerate(power_analysis['top_consumers'], 1):
        print(f"       {i}. {group['group']}: {group['total_power']*1000:.3f} mW ({group['percentage']:.1f}%)")

print("\n🏆 VLSI REPORT PARSING BENEFITS:")
print("✅ **Automated Metrics**: Extract key data from tool reports")
print("✅ **Trend Analysis**: Track metrics across design iterations")
print("✅ **Quality Gates**: Automated pass/fail checking")
print("✅ **Dashboard Ready**: Data suitable for visualization tools")

In [ ]:
# ERROR HANDLING AND ROBUST FILE OPERATIONS
# ==========================================
# Professional file handling with error management

print("🛡️ ERROR HANDLING AND ROBUST FILE OPERATIONS")
print("=" * 50)

import shutil
from datetime import datetime
import logging

# =============================================================================
# SAFE FILE OPERATIONS WITH ERROR HANDLING
# =============================================================================

print("\n🔒 SAFE FILE OPERATIONS WITH ERROR HANDLING:")

def safe_file_write(file_path, content, create_backup=True):
    """Safely write file with optional backup and error handling."""
    try:
        file_path = Path(file_path)

        # Create parent directories if needed
        file_path.parent.mkdir(parents=True, exist_ok=True)

        # Create backup if file exists
        if create_backup and file_path.exists():
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            backup_path = file_path.with_suffix(f".backup_{timestamp}{file_path.suffix}")
            shutil.copy2(file_path, backup_path)
            print(f"     📋 Backup created: {backup_path.name}")

        # Write new content
        with open(file_path, 'w', encoding='utf-8') as f:
            f.write(content)

        print(f"     ✅ File written: {file_path.name} ({len(content)} chars)")
        return True, None

    except PermissionError as e:
        error_msg = f"Permission denied: {file_path}"
        print(f"     ❌ {error_msg}")
        return False, error_msg
    except OSError as e:
        error_msg = f"OS error writing {file_path}: {e}"
        print(f"     ❌ {error_msg}")
        return False, error_msg
    except Exception as e:
        error_msg = f"Unexpected error: {e}"
        print(f"     ❌ {error_msg}")
        return False, error_msg

def safe_file_read(file_path, encoding='utf-8'):
    """Safely read file with comprehensive error handling."""
    try:
        file_path = Path(file_path)

        # Check if file exists
        if not file_path.exists():
            error_msg = f"File not found: {file_path}"
            print(f"     ❌ {error_msg}")
            return None, error_msg

        # Check if file is readable
        if not os.access(file_path, os.R_OK):
            error_msg = f"File not readable: {file_path}"
            print(f"     ❌ {error_msg}")
            return None, error_msg

        # Read file content
        with open(file_path, 'r', encoding=encoding) as f:
            content = f.read()

        print(f"     ✅ File read: {file_path.name} ({len(content)} chars)")
        return content, None

    except UnicodeDecodeError as e:
        error_msg = f"Encoding error reading {file_path}: {e}"
        print(f"     ❌ {error_msg}")
        return None, error_msg
    except Exception as e:
        error_msg = f"Error reading {file_path}: {e}"
        print(f"     ❌ {error_msg}")
        return None, error_msg

def validate_verilog_content(content):
    """Basic Verilog syntax validation."""
    errors = []
    lines = content.split('\n')

    # Check module/endmodule balance
    module_count = content.count('module')
    endmodule_count = content.count('endmodule')

    if module_count != endmodule_count:
        errors.append(f"Module/endmodule mismatch: {module_count} vs {endmodule_count}")

    # Check for basic syntax issues
    for i, line in enumerate(lines, 1):
        line_stripped = line.strip()
        if line_stripped and not line_stripped.startswith('//'):
            # Check for missing semicolons (simplified check)
            if any(keyword in line_stripped for keyword in ['reg ', 'wire ', 'input ', 'output ']):
                if not line_stripped.endswith(';') and not line_stripped.endswith(','):
                    if not any(char in line_stripped for char in ['(', ')', '[', ']']):
                        errors.append(f"Line {i}: Possible missing semicolon")

    return errors

def validate_sdc_content(content):
    """Basic SDC syntax validation."""
    errors = []
    lines = content.split('\n')

    # Check for valid SDC commands
    valid_commands = [
        'create_clock', 'create_generated_clock', 'set_input_delay', 'set_output_delay',
        'set_max_delay', 'set_min_delay', 'set_max_area', 'set_max_power',
        'set_clock_uncertainty', 'set_clock_latency'
    ]

    for i, line in enumerate(lines, 1):
        line_stripped = line.strip()
        if line_stripped and not line_stripped.startswith('#'):
            # Check if line starts with valid command
            if not any(line_stripped.startswith(cmd) for cmd in valid_commands):
                # Allow some common patterns
                if not any(pattern in line_stripped for pattern in ['set_', 'get_', '[', ']']):
                    errors.append(f"Line {i}: Unknown SDC command pattern")

    return errors

# Test safe file operations
with tempfile.TemporaryDirectory() as temp_dir:
    temp_path = Path(temp_dir)

    print("   Testing safe file operations:")

    # Test successful write and read
    test_verilog = """module test_counter (
    input wire clk,
    input wire reset_n,
    output reg [7:0] count
);

always @(posedge clk or negedge reset_n) begin
    if (!reset_n)
        count <= 8'h00;
    else
        count <= count + 1;
end

endmodule"""

    verilog_file = temp_path / "test_counter.v"
    success, error = safe_file_write(verilog_file, test_verilog)

    if success:
        content, error = safe_file_read(verilog_file)
        if content:
            # Validate content
            verilog_errors = validate_verilog_content(content)
            if verilog_errors:
                print(f"     ⚠️  Verilog validation errors: {verilog_errors}")
            else:
                print(f"     ✅ Verilog validation passed")

        # Test backup functionality
        modified_verilog = test_verilog.replace("count + 1", "count + 2")
        safe_file_write(verilog_file, modified_verilog, create_backup=True)

    # Test error handling - try to read non-existent file
    safe_file_read(temp_path / "nonexistent.v")

# =============================================================================
# FILE PROCESSING WITH LOGGING
# =============================================================================

print(f"\n📝 FILE PROCESSING WITH LOGGING:")

def setup_file_logger(name, log_file=None):
    """Set up logger for file operations."""
    logger = logging.getLogger(name)
    logger.setLevel(logging.INFO)

    # Remove existing handlers
    for handler in logger.handlers[:]:
        logger.removeHandler(handler)

    # Create formatter
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

    # Console handler
    console_handler = logging.StreamHandler()
    console_handler.setFormatter(formatter)
    logger.addHandler(console_handler)

    # File handler if specified
    if log_file:
        file_handler = logging.FileHandler(log_file)
        file_handler.setFormatter(formatter)
        logger.addHandler(file_handler)

    return logger

def process_design_files_with_logging(file_list, output_dir):
    """Process multiple design files with comprehensive logging."""
    logger = setup_file_logger("VLSIFileProcessor")

    logger.info(f"Starting batch processing of {len(file_list)} files")

    results = {
        'processed': 0,
        'errors': 0,
        'warnings': 0,
        'files': []
    }

    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)

    for file_info in file_list:
        file_name = file_info['name']
        file_content = file_info['content']
        file_type = file_info['type']

        logger.info(f"Processing {file_type} file: {file_name}")

        try:
            # Validate content based on type
            validation_errors = []
            if file_type == 'verilog':
                validation_errors = validate_verilog_content(file_content)
            elif file_type == 'sdc':
                validation_errors = validate_sdc_content(file_content)

            if validation_errors:
                logger.warning(f"Validation warnings for {file_name}: {validation_errors}")
                results['warnings'] += len(validation_errors)

            # Write file
            output_file = output_path / file_name
            success, error = safe_file_write(output_file, file_content, create_backup=False)

            if success:
                logger.info(f"Successfully processed {file_name}")
                results['processed'] += 1
                results['files'].append({
                    'name': file_name,
                    'status': 'success',
                    'size': len(file_content),
                    'warnings': len(validation_errors)
                })
            else:
                logger.error(f"Failed to process {file_name}: {error}")
                results['errors'] += 1
                results['files'].append({
                    'name': file_name,
                    'status': 'error',
                    'error': error
                })

        except Exception as e:
            logger.error(f"Unexpected error processing {file_name}: {e}")
            results['errors'] += 1

    logger.info(f"Batch processing complete: {results['processed']} success, {results['errors']} errors, {results['warnings']} warnings")
    return results

# Test file processing with logging
test_files = [
    {
        'name': 'cpu.v',
        'type': 'verilog',
        'content': 'module cpu(); endmodule'
    },
    {
        'name': 'memory.v',
        'type': 'verilog',
        'content': 'module memory(); reg [31:0] data; endmodule'
    },
    {
        'name': 'constraints.sdc',
        'type': 'sdc',
        'content': 'create_clock -period 10 clk\nset_input_delay 2 data'
    }
]

with tempfile.TemporaryDirectory() as temp_dir:
    print("   Processing files with logging:")
    results = process_design_files_with_logging(test_files, temp_dir)

    print(f"\n   Processing summary:")
    print(f"     Successful: {results['processed']}")
    print(f"     Errors: {results['errors']}")
    print(f"     Warnings: {results['warnings']}")

print("\n🏆 ROBUST FILE OPERATION BENEFITS:")
print("✅ **Error Recovery**: Graceful handling of file system errors")
print("✅ **Data Safety**: Automatic backups and validation")
print("✅ **Audit Trail**: Comprehensive logging of operations")
print("✅ **Production Ready**: Enterprise-grade file handling")
print("✅ **Quality Assurance**: Built-in validation and error checking")

## 💪 **Practice Exercises: File I/O Mastery**

### **🎯 Exercise 1: VLSI File Format Converter**
Create a comprehensive file format conversion system:
- Convert between Verilog and SPICE netlists
- Transform SDC constraints to different tool formats
- Handle different encoding and line ending formats
- Validate converted files for syntax correctness

### **🎯 Exercise 2: EDA Tool Log Analyzer**
Build a robust log analysis system:
- Parse multiple EDA tool log formats (synthesis, P&R, timing)
- Extract errors, warnings, and performance metrics
- Generate trend analysis from historical log data
- Create automated alerts for critical issues

### **🎯 Exercise 3: Design Metrics Dashboard Generator**
Implement a metrics processing and reporting system:
- Read timing, power, and area reports from multiple tools
- Aggregate data across different design corners
- Generate HTML/CSV reports with charts and tables
- Track metrics over design iterations

### **🎯 Exercise 4: Configuration Management Framework**
Create a robust configuration handling system:
- Validate JSON/YAML configuration files against schemas
- Support configuration inheritance and environment overrides
- Implement version control for configuration changes
- Generate configuration diffs and change reports

---

## 🏆 **Chapter Summary: File I/O Mastery Achieved**

### **✅ File Operation Fundamentals**
- **Text Processing**: Reading/writing Verilog, SDC, and report files
- **Structured Data**: CSV and JSON for metrics and configurations
- **Path Operations**: Cross-platform file and directory handling
- **File Validation**: Content checking and syntax validation

### **✅ Advanced Processing Techniques**
- **Report Parsing**: Automated extraction from timing and power reports
- **Error Handling**: Robust operations with comprehensive error management
- **Data Transformation**: Format conversion and content manipulation
- **Logging**: Professional audit trails and debugging support

### **✅ VLSI-Specific Applications**
- **Design Files**: Verilog module parsing and generation
- **Tool Integration**: Processing EDA tool inputs and outputs
- **Metrics Extraction**: Automated analysis of tool reports
- **Configuration Management**: Tool settings and design parameters

### **✅ Professional Features**
- **Backup Systems**: Automatic versioning and data safety
- **Validation**: Content verification and syntax checking
- **Performance**: Efficient processing of large files
- **Maintainability**: Clean, organized code with proper error handling

**🚀 Next**: Ready for Chapter 14: Advanced File Processing for Large-Scale VLSI Automation!